In [5]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [6]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using BenchmarkTools
using TransportMap
using QuadGK
using Polynomials
using Distributions
using Random
using LoopVectorization
using Optim
using NLsolve
using MLDataUtils
using MLDataPattern
using Test
using SparseArrays
using QRupdate
using AdaptiveTransportMap: vander, transform!, evaluate, ncoeff, optimize, negative_log_likelihood!, derivative
using Quadrature
using Cubature
# using Profile
# using ProfileView

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/AN696/src/RecipesBase.jl:117


### 1D example

In [219]:
Nx = 4
Ne = 500
X = randn(Nx, Ne)
ens = EnsembleState(X)
m = 10
B = MultiBasis(CstProHermite(3), Nx)

idx = [0 0 0 0; 0 2 0 1; 0 2 3 0; 2 0 2 1; 0 0 1 2; 1 2 0 2;3 2 2 2]

coeff =  randn(size(idx,1))

C = MapComponent(m, Nx, idx, coeff)
S = Storage(C.I.f, X);
R = C.I;

In [220]:
@time H = hess_x(C.I, X)

  0.001127 seconds (953 allocations: 7.733 MiB)


500×4×4 Array{Float64,3}:
[:, :, 1] =
 -0.10532     0.0  0.0   0.00518477
 -0.0823364   0.0  0.0  -0.018288
 -0.084217    0.0  0.0   0.0033643
 -0.0418507   0.0  0.0  -0.00720991
  0.0665937   0.0  0.0   0.0275316
  0.0153718   0.0  0.0  -0.0178412
  0.126472    0.0  0.0   0.0306717
 -0.086747    0.0  0.0   0.0230766
  0.134384    0.0  0.0  -0.00984135
  0.0130801   0.0  0.0   0.0062637
 -0.0370786   0.0  0.0   0.02353
  0.115376    0.0  0.0  -0.0267563
 -0.0562928   0.0  0.0  -0.0120922
  ⋮                     
  0.140134    0.0  0.0   0.0387605
 -0.0467703   0.0  0.0  -0.00823595
 -0.0736492   0.0  0.0   0.00894142
 -0.0551225   0.0  0.0   0.00424797
  0.00809251  0.0  0.0  -0.00105892
 -0.102967    0.0  0.0  -0.0253794
  0.0905319   0.0  0.0   0.0153992
  0.0365801   0.0  0.0  -0.0414786
  0.0612739   0.0  0.0   0.0270966
  0.0709517   0.0  0.0  -0.0318787
  0.110182    0.0  0.0  -0.0173336
 -0.055363    0.0  0.0  -0.0225103

[:, :, 2] =
 0.0   0.0659632   0.0   0.00328685
 0.0  -0.

In [200]:
ForwardDiff.hessian(x->R.f.f(x), member(ens, 1))

4×4 Array{Float64,2}:
 0.0915431    0.00750387   0.193767    0.0390633
 0.00750387   0.257496     0.127707   -0.0633798
 0.193767     0.127707    -0.462525   -0.0127379
 0.0390633   -0.0633798   -0.0127379   0.293287

In [201]:
ForwardDiff.jacobian(x->ForwardDiff.gradient(y->begin
            y[Nx] = 0.0
            return R.f.f(y) end, x), vcat(member(ens, 1)[1:end-1], 0.0))

4×4 Array{Float64,2}:
 0.0200897  0.0         0.215532  0.0
 0.0        0.0919115   0.130713  0.0
 0.215532   0.130713   -0.451732  0.0
 0.0        0.0         0.0       0.0

In [202]:
ForwardDiff.jacobian(x->ForwardDiff.gradient(y->begin
            y[Nx] = 0.0
            return R.f.f(y) end, x), member(ens, 1))

4×4 Array{Float64,2}:
 0.0200897  0.0         0.215532  0.0
 0.0        0.0919115   0.130713  0.0
 0.215532   0.130713   -0.451732  0.0
 0.0        0.0         0.0       0.0

In [203]:
ForwardDiff.hessian(x->begin
            x[Nx] = 0.0
            return R.f.f(x) end, member(ens, 1))

4×4 Array{Float64,2}:
 0.0200897  0.0         0.215532  0.0
 0.0        0.0919115   0.130713  0.0
 0.215532   0.130713   -0.451732  0.0
 0.0        0.0         0.0       0.0

In [204]:
H[1,:,:]

4×4 Array{Float64,2}:
 0.0578374   0.0         0.0          0.0200642
 0.0         0.177948    0.0         -0.0325539
 0.0         0.0        -0.457448    -0.00654263
 0.0200642  -0.0325539  -0.00654263   0.150642

In [205]:
ForwardDiff.gradient(y->begin 
        y[end] = 0.0
        return R.f.f(y) end, member(ens,1))

4-element Array{Float64,1}:
  0.08603063851938159
  0.046285984415219236
 -0.23409222602729426
  0.0

In [206]:
R.f.f(member(ens,1))

0.15926581110462468

In [207]:
ForwardDiff.gradient(z->R.g(ForwardDiff.gradient(y->R.f.f(y), z)[end]), member(ens,1))

4-element Array{Float64,1}:
  0.020064216152224094
 -0.032553934767634844
 -0.006542625847028468
  0.15064178521301513

In [208]:
Ht = zeros(Nx, Nx)
function Hessian!(H, x)
    fill!(H, 0.0)

    H[1:Nx-1,1:Nx-1] .= ForwardDiff.hessian(y->begin
                        y[Nx] = 0.0
                        return R.f.f(y) end, x)[1:Nx-1,1:Nx-1]

    # Hessian of the integral term
    H[1:Nx-1,1:Nx-1] .+= (quadgk(t -> ForwardDiff.hessian(
    z-> R.g(ForwardDiff.gradient(y->R.f.f(y), vcat(z[1:end-1], t))[end]), x),
            0.0, x[end], rtol = 1e-3)[1])[1:Nx-1,1:Nx-1]

    # H[Nx,:] and H[:,Nx]
    H[:, Nx] .= ForwardDiff.gradient(z->R.g(ForwardDiff.gradient(y->R.f.f(y), z)[end]), x)
    H[Nx, :] .= H[:, Nx]
    return H
end

Hessian! (generic function with 1 method)

In [221]:
@time Hessian!(Ht, member(ens,1))

  0.001482 seconds (1.66 k allocations: 901.766 KiB)


4×4 Array{Float64,2}:
 -0.10532     0.0125149    0.00796563   0.00518477
  0.0125149   0.0659632    0.0273109    0.00328685
  0.00796563  0.0273109   -0.179478     0.0177803
  0.00518477  0.00328685   0.0177803   -0.00669012

In [222]:
H[1,:,:]

4×4 Array{Float64,2}:
 -0.10532     0.0          0.0         0.00518477
  0.0         0.0659632    0.0         0.00328685
  0.0         0.0         -0.179478    0.0177803
  0.00518477  0.00328685   0.0177803  -0.00669012

### Test Hessian of the log π(x)

In [30]:
Nx = 2
Ne = 8
m = 10

X = randn(Nx, Ne)

idx = [0 0; 0 1; 1 0; 1 1; 1 2]
coeff =   [0.6285037650645056;
 -0.4744029092496623;
  1.1405280011620331;
 -0.7217760771894809;
  0.11855056306742319];

C = MapComponent(m, Nx, idx, coeff);

In [34]:
dψ_basis = grad_x_basis(C.I.f.f, X)

8×5×2 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  -0.207296    -0.130929     0.00117735
 0.0  0.0  -0.216523    -0.134105     0.0375507
 0.0  0.0   0.00618292   0.00360034   0.00205305
 0.0  0.0  -0.199714    -0.0830991   -0.107373
 0.0  0.0   0.209274     0.0991538   -0.106331
 0.0  0.0  -0.236677    -0.10036     -0.126702
 0.0  0.0  -0.270316    -0.120917     0.142049
 0.0  0.0  -0.0785179   -0.0495593    0.00259814

[:, :, 2] =
 0.0   0.00283979  0.0   0.00155384  0.345581
 0.0   0.0867129   0.0   0.0465448   0.31942
 0.0  -0.166026    0.0  -0.104855    0.307967
 0.0  -0.268816    0.0  -0.149159    0.0381479
 0.0   0.254048    0.0   0.138466    0.109751
 0.0  -0.26767     0.0  -0.064575    0.0207737
 0.0   0.262746    0.0   0.0959916   0.0506544
 0.0   0.0165449   0.0   0.0102847   0.391819

In [41]:
dxlogDJ = zeros(Ne, Nx)

ψoffdψxd = grad_xd(C.I.f.f, X)

8-element Array{Float64,1}:
  0.03850011681624252
 -0.036864339594153354
  0.19095459274793916
  0.23970874743411022
 -0.20745155095311177
  0.17605463803346075
 -0.18792702339659972
  0.03117818085903034

In [50]:
grad_x_grad_xd(C.I.f.f, X, C.I.f.f.idx)

8×2 Array{Float64,2}:
 -0.0150962    0.275081
 -0.00172344   0.272766
  0.00109835   0.191537
 -0.0403773   -0.00821251
 -0.0333779    0.127178
 -0.0481415    0.0110568
  0.0468207    0.0774556
 -0.00492957   0.294571

In [51]:
hess_xd(C.I.f.f, X)

8-element Array{Float64,1}:
  0.27508054698884865
  0.2727658334063561
  0.19153741860252596
 -0.008212514607585927
  0.12717812747627785
  0.01105679171823458
  0.07745559049870646
  0.29457111787580165

In [52]:
dxdxkψx_t = ForwardDiff.hessian(C.I.f.f, X[:,1])

2×2 Array{Float64,2}:
 -0.13341    -0.0150962
 -0.0150962   0.275081

In [ ]:
evaluate()

In [16]:
log_pdf(C, X)

8-element Array{Float64,1}:
 -1.890933371333304
 -1.0774421758804786
 -1.3812358207093065
 -2.756566903253647
 -2.2764943782683207
 -3.268114225886461
 -0.9741216255311081
 -1.442529469931936

In [18]:
Cx = evaluate(C, X)

8-element Array{Float64,1}:
 -1.3182090265735815
  0.5518744832820461
  1.0207836492619973
  1.969508900840876
  1.7095646000932772
  2.2193925767103297
  0.06731649217667446
  1.0856697228586412

In [22]:
getidx(C)

5×2 Array{Int64,2}:
 0  0
 0  1
 1  0
 1  1
 1  2

In [21]:
getcoeff(C)

5-element Array{Float64,1}:
  0.6285037650645056
 -0.4744029092496623
  1.1405280011620331
 -0.7217760771894809
  0.11855056306742319

In [23]:
dψ = grad_x_basis(C.I.f.f, X)

8×5×2 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0   0.107478    0.0190908  -0.043005
 0.0  0.0   0.0865389   0.0541193  -0.0107887
 0.0  0.0   0.138011    0.0855902   0.0231549
 0.0  0.0  -0.270698   -0.113793   -0.145219
 0.0  0.0  -0.259926   -0.128072   -0.127645
 0.0  0.0  -0.225594   -0.0869868  -0.122217
 0.0  0.0  -0.266312   -0.155186    0.0881003
 0.0  0.0   0.261878    0.157944    0.0678763

[:, :, 2] =
 0.0   0.200064   0.0   0.122532   -0.167233
 0.0   0.0623342  0.0   0.0386104   0.379666
 0.0  -0.0838875  0.0  -0.0502452   0.357862
 0.0  -0.268231   0.0  -0.105483    0.0306983
 0.0  -0.24554    0.0  -0.112246    0.113373
 0.0  -0.270878   0.0  -0.142266    0.00262786
 0.0   0.165408   0.0   0.0714071   0.211025
 0.0  -0.129595   0.0  -0.0583847   0.246624

In [24]:
d2ψ = hess_x_basis(C.I.f.f, X)

8×5×2×2 Array{Float64,4}:
[:, :, 1, 1] =
 0.0  0.0  -0.287372   -0.0510446    0.114986
 0.0  0.0  -0.297614   -0.18612      0.0371031
 0.0  0.0  -0.267679   -0.166006    -0.0449099
 0.0  0.0  -0.0102897  -0.00432547  -0.00552002
 0.0  0.0  -0.0807765  -0.0398007   -0.0396678
 0.0  0.0  -0.1657     -0.0638922   -0.089769
 0.0  0.0  -0.0515665  -0.030049     0.0170591
 0.0  0.0  -0.0730321  -0.0440471   -0.0189292

[:, :, 2, 1] =
 0.0   0.0   0.0   0.0215025   -0.0293467
 0.0   0.0   0.0   0.00539434   0.0530439
 0.0  -0.0   0.0  -0.0115774    0.0824581
 0.0  -0.0  -0.0   0.0726096   -0.0211313
 0.0  -0.0  -0.0   0.0638224   -0.0644633
 0.0  -0.0  -0.0   0.0611086   -0.00112877
 0.0   0.0  -0.0  -0.0440502   -0.130179
 0.0  -0.0   0.0  -0.0339381    0.143359

[:, :, 1, 2] =
 0.0   0.0   0.0   0.0215025   -0.0293467
 0.0   0.0   0.0   0.00539434   0.0530439
 0.0  -0.0   0.0  -0.0115774    0.0824581
 0.0  -0.0  -0.0   0.0726096   -0.0211313
 0.0  -0.0  -0.0   0.0638224   -0.0644633
 0.0  -

### Integration

In [20]:
Nx = 100
Ny = 50
m = 20
Ne = 400
X = randn(Nx, Ne).*randn(Nx, Ne) #.+ 0.5*randn(Nx).*cos.(randn(Nx, Ne) .* randn(Nx, Ne))

# L = LinearTransform(X)
# transform!(L, X);
# C = MapComponent(m, Nx)
# @time C, _ = optimize(C, X, 10)

In [24]:
M = HermiteMap(m, X);

In [25]:
@time M_noqr = optimize(M, X, 10; P = serial, start = Ny+1);

  9.550136 seconds (42.61 M allocations: 13.653 GiB, 7.59% gc time)


In [26]:
@time M_qr = optimize(M, X, 10; P = serial, withqr = true, start = Ny+1);

 18.997304 seconds (45.81 M allocations: 21.111 GiB, 4.17% gc time)


In [31]:
function timing()
    M = HermiteMap(m, X)
    @btime begin 
        optimize($M, $X, $10; P = serial, withqr = false, start = Ny+1)
    end
    
    @btime begin 
        optimize($M, $X, $10; P = serial, withqr = true, start = Ny+1)
    end
end

timing (generic function with 1 method)

In [32]:
timing();

  11.257 s (42606958 allocations: 13.65 GiB)
  21.346 s (45813192 allocations: 21.12 GiB)


In [11]:
timing();

  14.597 s (42433511 allocations: 13.37 GiB)


In [29]:
timing();

  973.450 ms (1054425 allocations: 1.43 GiB)


In [14]:
methods(update_component)

# 1 method for generic function "update_component":
[1] update_component(C::MapComponent, X, reduced_margin::Array{Int64,2}, S::Storage) in AdaptiveTransportMap at /media/mat/HDD/AdaptiveTransportMap/src/mapcomponent/greedyfit.jl:249

In [30]:
function timing()
    @btime begin 
        M = HermiteMap($m, $X)
        optimize($M, $X, $10; P = thread)
    end
end

timing (generic function with 1 method)

In [88]:
timing();

  297.730 ms (1000725 allocations: 1.35 GiB)


### Test optimization with QR

In [45]:
Nx = 3
Ne = 8
m = 20

idx = [0 0 0; 0 0 1; 0 1 0; 0 1 1; 0 1 2; 1 0 0]


Nψ = 6
coeff = [ 0.20649582065364197;
         -0.5150990160472986;
          2.630096893080717;
          1.13653076177397;
          0.6725837371023421;
         -1.3126095306624133]
C = MapComponent(m, Nx, idx, coeff; α = 1e-6);

Ne = 100


# The QR decomposition is not unique!

X = randn(Nx, Ne) .* randn(Nx, Ne) + cos.(randn(Nx, Ne)) .* exp.(-randn(Nx, Ne).^2)

L = LinearTransform(X)
transform!(L, X)
S = Storage(C.I.f, X)
F = QRscaling(S)
newidx = [1 1 1]

Snew = update_storage(S, X, newidx)
Fupdated = updateQRscaling(F, Snew)

Fnew = QRscaling(Snew)

@test norm(Fupdated.D - Fnew.D)<1e-8
@test norm(Fupdated.Dinv - Fnew.Dinv)<1e-8

Test Passed